In [1]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
cal_len = 5

In [3]:
chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
num_classes = len(chars)

char_to_index = {char: idx for idx, char in enumerate(chars)}
index_to_index = {idx: char for idx, char in enumerate(chars)}

In [4]:
# 학습 데이터 생성
train_data = np.random.choice(list(chars), size = (10000, cal_len))
label_data = np.flip(train_data, axis = 1)

In [5]:
# 문자 데이터를 숫자 인덱스로 변환
train_data_indices = np.array([[char_to_index[char] for char in seq] for seq in train_data])
label_data_indices = np.array([[char_to_index[char] for char in seq] for seq in label_data])

print(train_data[:4])
print(label_data[:4])

[['j' 'H' 'T' 'F' 'W']
 ['L' 'y' 'H' 'M' 'I']
 ['A' 'B' 'Z' 'j' 'f']
 ['m' 'e' 'v' 'K' 'O']]
[['W' 'F' 'T' 'H' 'j']
 ['I' 'M' 'H' 'y' 'L']
 ['f' 'j' 'Z' 'B' 'A']
 ['O' 'K' 'v' 'e' 'm']]


In [6]:
# pad_sequence 전처리
padded_train = pad_sequences(train_data_indices, maxlen = cal_len)
padded_labels = pad_sequences(label_data_indices, maxlen = cal_len)

In [7]:
# 모델링
model = Sequential()
model.add(Embedding(num_classes, 100, input_length = cal_len))
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Dense(num_classes, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 100)            5200      
                                                                 
 bidirectional (Bidirectiona  (None, 5, 128)           84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 5, 52)             6708      
                                                                 
Total params: 96,388
Trainable params: 96,388
Non-trainable params: 0
_________________________________________________________________


In [8]:
# 모델 컴파일
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [9]:
# 모델 실행
history = model.fit(padded_train, padded_labels, epochs = 100, batch_size = 32, validation_split = 0.2)

Epoch 1/100
250/250 [==============================] - 19s 41ms/step - loss: 3.0537 - accuracy: 0.2585 - val_loss: 1.9173 - val_accuracy: 0.4354
Epoch 2/100
250/250 [==============================] - 7s 28ms/step - loss: 1.6078 - accuracy: 0.4615 - val_loss: 1.4693 - val_accuracy: 0.4742
Epoch 3/100
250/250 [==============================] - 6s 26ms/step - loss: 1.3369 - accuracy: 0.4983 - val_loss: 1.3060 - val_accuracy: 0.5022
Epoch 4/100
250/250 [==============================] - 7s 26ms/step - loss: 1.2045 - accuracy: 0.5321 - val_loss: 1.2133 - val_accuracy: 0.5301
Epoch 5/100
250/250 [==============================] - 7s 27ms/step - loss: 1.1109 - accuracy: 0.5613 - val_loss: 1.1457 - val_accuracy: 0.5431
Epoch 6/100
250/250 [==============================] - 6s 26ms/step - loss: 1.0339 - accuracy: 0.5888 - val_loss: 1.0885 - val_accuracy: 0.5680
Epoch 7/100
250/250 [==============================] - 5s 22ms/step - loss: 0.9706 - accuracy: 0.6140 - val_loss: 1.0355 - val_accuracy

In [10]:
# loss와 accuracy
loss, acc = model.evaluate(padded_train, padded_labels)
print(f"loss : {loss: .4f}, acc = {acc: .4f}")

313/313 [==============================] - 3s 11ms/step - loss: 0.0123 - accuracy: 0.9965
loss :  0.0123, acc =  0.9965


In [11]:
# 랜덤한 영문 대소문자 5글자를 요소로 하는 20개 리스트 생성
test_data = ["".join(np.random.choice(list(chars), cal_len)) for _ in range(20)]

In [12]:
# 거꾸로된 리스트 생성
expected_outputs = [s[::-1] for s in test_data]

In [13]:
correct_predictions = 0
total_predictons = len(test_data)

for i, test_string in enumerate(test_data):
    numbers = [char_to_index[char] for char in test_string]
    array_numbers = np.array([numbers])
    padded_numbers = pad_sequences(array_numbers, maxlen = cal_len)
    prediction = model.predict(padded_numbers)
    predicted_indices = np.argmax(prediction, axis = -1)[0]
    predicted_chars = [index_to_index[idx] for idx in predicted_indices]
    predicted_output = "".join(predicted_chars)

    is_correct = predicted_output == expected_outputs[i]
    if is_correct:
        correct_predictions += 1
    print(f"입력: {test_string}")
    print(f"예측된 출력:  {predicted_output}")
    print(f"실제 출력: {expected_outputs[i]}")
    print(f"정확 여부: {'맞음' if is_correct else '틀림'}\n")

accuracy = correct_predictions / total_predictons
print(f"총 정확도:  {accuracy * 100:.2f}%")

1/1 [==============================] - 1s 1s/step
입력: oyCSW
예측된 출력:  WSCyo
실제 출력: WSCyo
정확 여부: 맞음

1/1 [==============================] - 0s 38ms/step
입력: LuWSi
예측된 출력:  iSWuL
실제 출력: iSWuL
정확 여부: 맞음

1/1 [==============================] - 0s 40ms/step
입력: KHakt
예측된 출력:  tkaHK
실제 출력: tkaHK
정확 여부: 맞음

1/1 [==============================] - 0s 47ms/step
입력: NFlnX
예측된 출력:  XnlFN
실제 출력: XnlFN
정확 여부: 맞음

1/1 [==============================] - 0s 49ms/step
입력: UhMgS
예측된 출력:  SgMhU
실제 출력: SgMhU
정확 여부: 맞음

1/1 [==============================] - 0s 45ms/step
입력: qURKO
예측된 출력:  OKRUq
실제 출력: OKRUq
정확 여부: 맞음

1/1 [==============================] - 0s 64ms/step
입력: mjJAO
예측된 출력:  OAJjm
실제 출력: OAJjm
정확 여부: 맞음

1/1 [==============================] - 0s 67ms/step
입력: WMlDf
예측된 출력:  fDlMW
실제 출력: fDlMW
정확 여부: 맞음

1/1 [==============================] - 0s 47ms/step
입력: BHrjU
예측된 출력:  UjrHB
실제 출력: UjrHB
정확 여부: 맞음

1/1 [==============================] - 0s 44ms/step
입력: nwjIH
예측된 출력:  HIjwn
실제 출력: HIjwn
정확 